# Translation of Elisabeth's hot spot Javascript pipeline to Python

NOTE: In Ouagadougou (Burkina Faso), the "rural" area is bare laterite soil and dry brush. In the hot season, this heats up faster than the city. We should probably use a different approach to Hoang et al. 2025, since the rural areas are actualy hotter than the urban areas. We might consider, for example, using an intra-urban heat comparison between pixels (Lindén, 2011).


Modifications:
- Urban area is just a point and buffer, we need to use the administrative boundary file (from Elisabeth)
- Fixed the cloud masking (should keep pixels where the mask is 1)
- Included both landsat 8 and 9 for better temporal coverage (8-day revisit time gives us roughly 32 images per year vs. 16 images per year)
- Used both 2023 and 2024 to increase number of images
- Focused only on hot months (since there's large variability between hot and winters seasons)
- Removed images with heavy cloud/dust, since heavy dust absorbs surface heat and radiates it at a different temperature, distorting LST readings
- 

Lindén, J. (2011). Nocturnal Cool Island in the Sahelian city of Ouagadougou, Burkina Faso. International Journal of Climatology, 31.



In [25]:
import ee
import geemap
import geemap.colormaps as cm


# Initialize Earth Engine
ee.Initialize()


# ==========================================================
# 1. LOAD URBAN AREA (OUAGADOUGOU)
# ==========================================================
# Temporarily using lat/lon point with a buffer (should use Elisabeth's Ouaga boundary)
urban = ee.Geometry.Point([-1.5197, 12.3714]).buffer(15000) 

# Note: For the asset, use .geometry() in export/reduceRegion
# urban_fc = ee.FeatureCollection("projects/hotspotters/assets/Ouaga_boundary")
# urban = urban_fc.geometry() 


# ==========================================================
# 2. LOAD LANDSAT-8 & 9 IMAGES AND COMPUTE LST (°C)
# ==========================================================
# Helper function to process images
def process_landsat(img):
    # Scale factors for Collection 2 Level 2
    # ST_B10 is Surface Temperature in Kelvin. Scale: 0.00341802, Offset: 149.0
    lst_kelvin = img.select('ST_B10').multiply(0.00341802).add(149.0)
    
    # Convert to Celsius
    lst_celsius = lst_kelvin.subtract(273.15).rename('LST')
    
    # QA Masking (Bit 6 is 'Clear', Bit 8 is 'Cloud Shadow', Bit 9 is 'Cloud')
    # We strictly want clear pixels
    # 1 << 6 checks bit 6 and `.neq(0)` means "is not zero" (i.e., is 1)
    qa = img.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1 << 6).neq(0) 
    
    return lst_celsius.updateMask(mask).copyProperties(img, ['system:time_start'])

# Load images (using both Landsat 8 and 9 give better temporal coverage)
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")

# Merge and process landsat collections
landsat_collection = (
    l8.merge(l9)
    .filterBounds(urban)
    .filterDate('2023-03-01', '2024-05-30') # Use a range 2021-2024 to reduce cloud gaps
    .filter(ee.Filter.calendarRange(3, 5, 'month'))  # Filter for the HOT SEASON (March - May for Ouagadougou; we only want to map heat stress, not rainy season cooling)
    .filter(ee.Filter.lt('CLOUD_COVER', 20))    # Filter scenes to remove heavy cloud/dust contamination and follow Hoang et al. methods
    .map(process_landsat)
    )

# Create seasonal median composite
lst_median = landsat_collection.median().clip(urban)


# ==========================================================
# 4. COMPUTE CITY-WIDE MEAN AND STANDARD DEVIATION
#    (Used for statistical hotspot thresholding)
# ==========================================================
stats = lst_median.reduceRegion(
    reducer=ee.Reducer.mean().combine(
        reducer2=ee.Reducer.stdDev(),
        sharedInputs=True
    ),
    geometry=urban,
    scale=30,
    maxPixels=1e13,
    bestEffort=True
)

stats_local = stats.getInfo()
mean_lst = stats_local['LST_mean']
std_lst = stats_local['LST_stdDev']
print(f'Mean LST: {mean_lst:.2f} °C')
print(f'Std Dev:  {std_lst:.2f} °C')


# ==========================================================
# 5. DEFINE HOTSPOT THRESHOLD = Mean + 1 * StdDev
# ==========================================================
threshold = mean_lst + std_lst
print(f'Hotspot Threshold: {threshold:.2f} °C')


# ==========================================================
# 6. EXTRACT HOTSPOTS (BINARY MAP)
#    Hotspot pixels = LST > threshold
# ==========================================================
hotspots = lst_median.gt(threshold).selfMask()


# ==========================================================
# OPTIONAL: VISUALIZE RESULTS
# ==========================================================
Map = geemap.Map()
Map.centerObject(urban, 11)
# Style and visualization settings
palette = cm.get_palette('inferno')
vis_params = {
    'min': 25, 
    'max': 55,  # Adjusted max to 55 for Ouaga hot season intensity
    'palette': palette
}
# Display LST + colorbar
Map.addLayer(lst_median, vis_params, 'Median LST (Hot Season)')
Map.add_colorbar(vis_params=vis_params, label='LST (°C)')
# Display Hotspots + legend
red_hex = "FF2400"
Map.addLayer(hotspots, {'palette': [red_hex]}, 'UHI Hotspots (> Mean + 1SD)')
Map.add_legend(legend_dict={'UHI Hotspots\n(LST > Mean + 1SD)': red_hex}, 
               title='Urban Heat Islands', position='bottomright')
Map

# ==========================================================
# 7. EXPORT HOTSPOT MAP (GeoTIFF)
# ==========================================================
# # task = ee.batch.Export.image.toDrive(
#     image=hotspots,
#     description='Ouagadougou_Hotspots_2020_2024_HotSeason',
#     region=urban,
#     scale=30,
#     maxPixels=1e13
# )
# task.start()

Mean LST: 45.95 °C
Std Dev:  2.17 °C
Hotspot Threshold: 48.12 °C


Map(center=[12.371415388278361, -1.5196995631175392], controls=(WidgetControl(options=['position', 'transparen…